In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import html5lib

In [4]:
def makeSoup(url):
    response=requests.get(url)
    soup=BeautifulSoup(response.content,"lxml")
    return soup

In [57]:
def dataCollector(soup):
    import re
    # make the soup
    this = makeSoup(soup)
    
    # find the right table
    tables = this.findChildren('table')
    table = tables[2]
    
    # turn the data into a workable format
    data   = [[td.text for td in row.select('td')]
             for row in table.findAll('tr')]
    
    
    body = data[1:][1:]
    cols   =  zip(*body)
    
    # create a dict with the data
    #tbl_d  = {name:col for name, col in zip(header,cols)}
    
    return body

In [58]:
dataCollector('https://en.wikipedia.org/wiki/The_Bachelorette_(season_2)')

[['Aaron', 'Harold', 'Rick', 'Ian', 'Chad', 'Ian', 'Matthew', 'Ian'],
 ['Andy', 'Todd', 'Todd', 'Sean', 'Ian', 'Matthew', 'Ian', 'Matthew'],
 ['Anselm', 'Brad', 'Chad', 'Chad', 'Lanny', 'Chad', 'Chad', ''],
 ['Brad', 'Ryan M.', 'Brad', 'Ryan M.', 'Matthew', 'Lanny', ''],
 ['Brian', 'Ian', 'Sean', 'Brad', 'Brad', ''],
 ['Chad', 'Chad', 'Lanny', 'Matthew', 'Ryan M.', ''],
 ['Chris', 'Lanny', 'Ryan R.', 'Lanny', 'Sean', ''],
 ['Cory', 'Robert', 'Matthew', 'Rick', ''],
 ['Damon', 'Sean', 'Ian', 'Ryan R.', ''],
 ['Eliot', 'Ryan R.', 'Ryan M.', 'Todd', ''],
 ['Harold', 'Damon', 'Damon', ''],
 ['Ian', 'Eliot', 'Eliot', ''],
 ['Jeff', 'Matthew', 'Harold', ''],
 ['Justin', 'Marcus', 'Marcus', ''],
 ['Keith', 'Rick', 'Robert', ''],
 ['Lanny', 'Aaron', ''],
 ['Marcus', 'Andy', ''],
 ['Matthew', 'Anselm', ''],
 ['Rick', 'Brian', ''],
 ['Robert', 'Chris', ''],
 ['Ryan M.', 'Cory', ''],
 ['Ryan R.', 'Jeff', ''],
 ['Sean', 'Justin', ''],
 ['Todd', 'Keith', ''],
 ['Trever', 'Trever', '']]

In [ ]:
def dataCollector(soup):
    import re
    # make the soup
    this = makeSoup(soup)
    
    # find the right table
    tables = this.findChildren('table')
    table = tables[1]
    
    # turn the data into a workable format
    data   = [[td.text for td in row.select('td')]
             for row in table.findAll('tr')]
    
    # create the header row and the body
    header = ['name','age', 'hometown', 'occupation', 'elimination']
    body = data[1:]
    cols   =  zip(*body)
    
    # create a dict with the data
    tbl_d  = {name:col for name, col in zip(header,cols)}
    
    # extract the season number from the original soup
    number = re.findall(r'\d+', soup)
    
    # remove brackets
    num= ''.join(number)
    
    # create a new key of seasons
    tbl_d['season'] = [num] * len(tbl_d['age'])
    
    new_names = []
    for name in tbl_d['name']:
        cleaned_name = re.sub(r'\[\w+\]', ' ', name)
        new_names.append(cleaned_name)

    tbl_d['name'] = new_names
    
    # return dictionary
    return tbl_d